In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import torch
import gloria
import pandas as pd 
import numpy as np 

df = pd.read_csv('pretrained/chexpert_5x200.csv')
df['Path'] = '../imagedata/' + df['Path']


2023-11-26 21:53:12.190002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 21:53:13.028093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
cls_prompts_list = np.load('cls_prompts_5.npy', allow_pickle=True)

In [8]:
# load model
device = "cuda:3" if torch.cuda.is_available() else "cpu"

# gloria_model = gloria.load_gloria(name = 'data/ckpt/gloria_pretrain_trial_1_1.0/2023_11_22_22_52_17/epoch=14-step=12149.ckpt', device=device) # resnet50
gloria_model = gloria.load_gloria(name = 'data/ckpt/gloria_pretrain_trial_1_1.0/2023_11_23_10_53_14/epoch=16-step=13769.ckpt', device=device) # vit-hybrid

gloria_model = gloria_model.eval()

In [5]:
gt = np.concatenate([np.array([0]*200), np.array([1]*200),np.array([2]*200), np.array([3]*200), np.array([4]*200)])

## Zero-shot classification

In [9]:
acc_list = []

for i, cls_prompts in enumerate(cls_prompts_list):
    print (f'Run: {i}')
    
    processed_imgs = gloria_model.process_img(df['Path'].tolist(), device) 

    processed_txt = gloria_model.process_class_prompts(cls_prompts, device)

    with torch.no_grad():
        # zero-shot classification on 1000 images
        similarities = gloria.zero_shot_classification(
            gloria_model, processed_imgs, processed_txt)

    # print(similarities)
    sim = similarities.to_numpy()
    pred = np.argmax(sim, axis=1)
    acc = (pred==gt).sum()/len(gt)
    
    acc_list.append(acc)
    print (f'Accuracy: {acc}')
    



Run: 0


/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))


Accuracy: 0.359
Run: 1


/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))


Accuracy: 0.393
Run: 2


...

Accuracy: 0.437
Run: 3


/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))


Accuracy: 0.44
Run: 4


/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))


Accuracy: 0.343


In [10]:
print (f'Avg acc: {np.mean(acc_list)}')
print (f'Std acc: {np.std(acc_list)}')

Avg acc: 0.3944
Std acc: 0.03947454876246212


## Zero-shot classification for large model 


In [11]:
gloria_model = gloria.load_gloria(name = 'data/ckpt/gloria_pretrain_trial_1_1.0/2023_11_23_19_58_12/epoch=14-step=12149.ckpt', device=device) # swin

gloria_model = gloria_model.eval()


In [12]:
from gloria.utils import utils

In [13]:
acc_list = []

for idx, cls_prompts in enumerate(cls_prompts_list):
    print (f'Run: {idx}')

    similarities_list = []
    
    processed_txt = gloria_model.process_class_prompts(cls_prompts, device)

    for i in range(0,1000,100):
        processed_imgs = gloria_model.process_img(df['Path'].tolist()[i:i+100], device) 

        with torch.no_grad():
            
            similarities_sample = gloria.zero_shot_classification_no_norm(
                gloria_model, processed_imgs, processed_txt)

        similarities_list.append(similarities_sample)
    
    
    similarities = np.concatenate(similarities_list, axis=0)
    sim = utils.normalize(similarities)
    
    pred = np.argmax(sim, axis=1)
    acc = (pred==gt).sum()/len(gt)
    
    acc_list.append(acc)
    print (f'Accuracy: {acc}')
    


Run: 0


/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/glor

Accuracy: 0.335
Run: 1


/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/glor

Accuracy: 0.28
Run: 2


...

Accuracy: 0.391
Run: 3


/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/glor

Accuracy: 0.367
Run: 4


/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/gloria/models/gloria_model.py:271: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_imgs.append(torch.tensor(img))
/raid/biplab/phduser1/saikat/ADL_project/gloria/glor

Accuracy: 0.392


In [17]:
acc_list

[0.335, 0.28, 0.391, 0.367, 0.392]

In [18]:
print (f'Avg acc: {np.mean(acc_list)}')
print (f'Std acc: {np.std(acc_list)}')

Avg acc: 0.35300000000000004
Std acc: 0.041985711855344304
